In [1]:
import os
import pandas as pd

In [2]:
### still going to need to make new dir for copies ###

data_dir = "./Data/EMG_data_for_gestures-master/"
subj_nums = os.listdir(data_dir)
subj_data = {n : os.listdir(data_dir + n) for n in subj_nums}

In [3]:
for n in subj_nums:
    print(os.listdir(data_dir + n + "/"))

['2_raw_data_09-50_21.03.16.txt', '1_raw_data_09-49_21.03.16.txt']
['1_raw_data_10-38_11.04.16.txt', '2_raw_data_10-40_11.04.16.txt']
['2_raw_data_12-43_23.03.16.txt', '1_raw_data_12-41_23.03.16.txt']
['2_raw_data_09-34_11.04.16.txt', '1_raw_data_09-32_11.04.16.txt']
['1_raw_data_10-28_30.03.16.txt', '2_raw_data_10-29_30.03.16.txt']
['1_raw_data_20-28_24.04.16.txt', '2_raw_data_20-30_24.04.16.txt']
['2_raw_data_11-20_23.03.16.txt', '1_raw_data_11-19_23.03.16.txt']
['2_raw_data_13-13_22.03.16.txt', '1_raw_data_13-12_22.03.16.txt']
['2_raw_data_12-14_25.04.16.txt', '1_raw_data_12-12_25.04.16.txt']
['2_raw_data_10-23_29.03.16.txt', '1_raw_data_10-22_29.03.16.txt']
['2_raw_data_12-06_27.04.16.txt', '1_raw_data_12-04_27.04.16.txt']
['2_raw_data_18-03_24.04.16.txt', '1_raw_data_18-02_24.04.16.txt']
['2_raw_data_12-16_23.03.16.txt', '1_raw_data_12-14_23.03.16.txt']
['2_raw_data_12-20_06.04.16.txt', '1_raw_data_12-19_06.04.16.txt']
['1_raw_data_09-50_15.04.16.txt', '2_raw_data_09-51_15.04.16.t

In [4]:
# looking at this subject
tst_subj = "03"

In [5]:
tst_path_0 = data_dir + tst_subj + "/" + subj_data[tst_subj][0]
tt0 = pd.read_table(tst_path_0)

In [6]:
tst_path_1 = data_dir + tst_subj + "/" + subj_data[tst_subj][1]
tt1 = pd.read_table(tst_path_1)

In [7]:
tst_path_1 # example path

'./Data/EMG_data_for_gestures-master/03/1_raw_data_09-32_11.04.16.txt'

In [8]:
# rename "time" variable to "time_ms"
# rename "class" variable to "gesture"
colnames = list(tt0.columns)
colnames[0] = "time_ms"
colnames[-1] = "gesture"
tt0.columns = colnames
tt1.columns = colnames

In [9]:
def mod_col_names(full_frame):
    """modify column names of full_frame prior to any other manipulation"""
    # rename "time" variable to "time_ms"
    # rename "class" variable to "gesture"
    colnames = list(full_frame.columns)
    colnames[0] = "time_ms"
    colnames[-1] = "gesture"
    full_frame.columns = colnames
    return full_frame

In [10]:
def find_break_pts(gesture, t = 1000):
    """find and return index for moments in data frame with > t ms between pt and next pt"""
    nrow = gesture.shape[0]
    brk_pts = [i-1 for i in range(2, nrow) if gesture.iloc[i].time_ms - gesture.iloc[i-1].time_ms > t]
    return brk_pts

In [11]:
def zero_time(seg_df):
    """take previously segmented DataFrame by gesture and set time_ms to start at 0"""
    end_time = min(seg_df.time_ms)
    seg_df.time_ms = seg_df.time_ms - end_time
    return seg_df

In [12]:
def gest_dict(filepath):
    """return dict where key is gesture # and value is indiv DataFrames for that gesture"""
    df = pd.read_table(filepath)
    df = mod_col_names(df)
    gests = df.gesture.unique()
    all_gst = dict()
    # seperate gestures in file
    for g in gests:
        # split single gesture df into iterations of gesture g
        brk_pts = find_break_pts(df[df.gesture == g])
        gd = dict() # store sub-dfs created by breaks
        trail = 0 
        for n, b in enumerate(brk_pts):
            if g == 0:
                # treat gest == 0 seperately
                # due to many cases btwn non-0 gests
                gd[n] = df[df.gesture == 0].iloc[trail:b]
                gd[n] = zero_time(gd[n])
                train = b + 1
            else:
                # this will handle all other cases (gesture != 0)
                gd[0] = zero_time(df[df.gesture == g].iloc[:b])
                gd[1] = zero_time(df[df.gesture == g].iloc[b+1:])
            all_gst[g] = gd
    
    return all_gst

In [21]:
def build_new_ref(to_dir, sn, orig_file, g, ct):
    """build a new file path based on attributes of data to be saved; naming convention:
    to_dir - top level dir for cleaned data
    sn - subject number
    orig_file - file name with raw data; 2 per subject
    g - gesture number
    ct - counter for files w/ similar names (e.g. gesture 2, 0 & 1)
    """
    if orig_file.startswith("1"):
        new_ref = to_dir + sn + "_1_" + str(g) + "_" + str(ct) + ".csv"
    else:
        new_ref = to_dir + sn + "_2_" + str(g) + "_" + str(ct) + ".csv"
    return new_ref

In [14]:
tst = gest_dict(tst_path_1)[6]

In [15]:
tst_path_1

'./Data/EMG_data_for_gestures-master/03/1_raw_data_09-32_11.04.16.txt'

In [16]:
for j in subj_nums:
    # get files 1 and 2 for subject j
    fp0 = data_dir + j + "/" + subj_data[j][0]
    fp1 = data_dir + j + "/" + subj_data[j][1]
    gest_dict(fp0)

In [17]:
# check if residual movement from gesture persists to 0 range

In [18]:
to_dir = "./Data/EMG_data_for_gestures-cleaned/"
build_new_ref(to_dir, "30", "origin", "5", 3)

'./Data/EMG_data_for_gestures-cleaned/30/2/5_3'

In [ ]:
tst[0]

In [ ]:
raw_dir = "./Data/EMG_data_for_gestures-master/"
# generate new data files
for n, v in subj_data.items():
    # origin file 1 or 2 for each subject
    for o_f in (0, 1):
        cln_gests = gest_dict(raw_dir + n + "/" + v[o_f])
        # parse data file seperating frames and saving them to new dir
        for g, d in cln_gests.items():
            ct = 0 # counter for new file names
            ### need another level here for d - see jupyter notebook
            clean_ref = build_new_ref(to_dir, n, v[o_f], g, ct)
            print(d)

In [24]:
tdf = tst[0]

In [23]:
?tdf.to_csv

In [27]:
tst.values()

dict_values([       time_ms  channel1  channel2  channel3  channel4  channel5  channel6  \
26348        0   0.00008   0.00010   0.00011  -0.00001  -0.00019  -0.00046   
26349        1   0.00008   0.00010   0.00011  -0.00001  -0.00019  -0.00046   
26350        2   0.00008   0.00010   0.00011  -0.00001  -0.00019  -0.00046   
26351        3   0.00008   0.00010   0.00011  -0.00001  -0.00019  -0.00046   
26352        4   0.00008   0.00010   0.00011  -0.00001  -0.00019  -0.00046   
26353        5   0.00008   0.00010   0.00011  -0.00001  -0.00019  -0.00046   
26354        6   0.00008   0.00010   0.00011  -0.00001  -0.00019  -0.00046   
26355        8   0.00008   0.00010   0.00011  -0.00001  -0.00019  -0.00046   
26356        9   0.00008   0.00010   0.00011  -0.00001  -0.00019  -0.00046   
26357       10   0.00008   0.00010   0.00011  -0.00001  -0.00019  -0.00046   
26358       11   0.00008   0.00010   0.00011  -0.00001  -0.00019  -0.00046   
26359       12   0.00008   0.00010   0.00011  -0.00